In [8]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pytesseract
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import base64
import time
from PIL import Image
from io import BytesIO
import warnings
warnings.filterwarnings("ignore")

def cropImg(img):
    location = img.location
    size = img.size
    top,bottom,left,right = location['y'], location['y']+size['height'],\
                            location['x'], location['x']+size['width']
    screenshot = driver.get_screenshot_as_png()
    screenshot = Image.open(BytesIO(screenshot))
    screenshot = screenshot.crop((left, top, right, bottom))
    screenshot.save('/Users/syuan/captcha/captcha_login.png')
    img1 = cv2.imread("/Users/syuan/captcha/captcha_login.png", cv2.IMREAD_GRAYSCALE)
    return img1

def pre_precessing(img):
#     plt.figure(figsize=(12, 14))
    col = len(img[0])
    row = len(img)
    #在圖的外圍加一圈pixel，防止index error。
    b = np.array([[255] for i in range(45)])
    new_arr = np.concatenate(img, b, axis=1)
    new_arr2 = np.array([np.insert(row, 0, 255) for row in new_arr])
    c = [[255]*142]
    new_arr3 = np.concatenate(new_arr2, c, axis=0)
    dst = np.vstack((c, new_arr3)).astype(dtype='uint8')

    #觀察到圖片噪點幾乎都是單一pixel，且也有橫/斜線，因此遍歷圖上所有pixel，若上下或左右皆是白色，則將此pixel設為白色
    for i in range(1, row):
        for j in range(1, col):
            if dst[i-1][j] == 255 and dst[i+1][j] == 255:
                dst[i][j] = 255
            elif dst[i][j-1] == 255 and dst[i][j+1] == 255:
                dst[i][j] = 255

#     plt.imshow(dst, cmap='gray')
    blur_img = cv2.GaussianBlur(dst, (0, 0), 100)#先用高斯模糊
    usm = cv2.addWeighted(dst, 0.5, blur_img, -0.5, 0)#以原圖:模糊圖片=0.5:-0.5的比例進行混合，得到銳化後的圖片
#     plt.figure(figsize=(5, 6))
#     plt.title('sharpening')
#     plt.imshow(usm, cmap='gray')
    dst = np.array(usm, dtype = np.uint8)
    capcha = pytesseract.image_to_string(dst, lang = 'eng')
    regex = re.compile("\d*\d*\d*\d*\d*")
    match = regex.findall(capcha)
    return ''.join(match), dst

def cut_img(p_img):
    temp = []
    for j in range(5):
        #plt.subplot(1, 5, j+1)
        img = p_img[10:40, 10+(j)*25:40+(j)*25]#分割數字
        temp.append(img)
        #plt.imshow(temp[-1], cmap='gray')
    return temp

In [ ]:
driver = webdriver.Chrome("/Users/syuan/chromedriver_mac64/chromedriver")
driver.get("https://reg-prod.tzuchi-healthcare.org.tw/tchw/HIS5OpdReg/RegQryCancel?Loc=XD")

In [513]:
X_train = pd.Series([])
y_train = pd.Series([], dtype='uint8')
for i in range(1000):
    img = driver.find_element(By.XPATH, "//*[@id='MainContent_imgVCode']")#定位驗證碼
    capcha = cropImg(img)#網頁截圖並切割出驗證碼
    string, p_img= pre_precessing(capcha)#OCR辨識
    #輸入資料
    driver.find_element(By.XPATH, "//*[@id='MainContent_tbxMRNo']").send_keys(id_number)
    driver.find_element(By.XPATH, "//*[@id='MainContent_tbxBirthday']").send_keys(birthday)
    driver.find_element(By.XPATH, "//*[@id='MainContent_tbxVCode']").send_keys(string)
    #點擊查詢
    driver.find_element(By.XPATH, "//*[@id='MainContent_btnQry']").click()
    time.sleep(0.5)
    try:
        #若成功則儲存驗證碼及其標籤
        driver.find_element(By.XPATH, "//*[@id='MainContent_gvList']/tbody/tr/td")
        
        c_img = pd.Series(cut_img(p_img))
        lst_num = [int(i) for i in string]
        series_num = pd.Series(lst_num, dtype='uint8')
        X_train = X_train.append(c_img)
        y_train = y_train.append(series_num)
            
        driver.back()
        time.sleep(0.5)
        actions = ActionChains(driver)
        actions.send_keys(Keys.ENTER)
        driver.find_element(By.XPATH, "//*[@id='MainContent_tbxMRNo']").clear()
        driver.find_element(By.XPATH, "//*[@id='MainContent_tbxBirthday']").clear()
        driver.find_element(By.XPATH, "//*[@id='MainContent_tbxVCode']").clear()
        driver.find_element(By.XPATH, "//*[@id='MainContent_btnReflash']").click()
    except:
        actions = ActionChains(driver)
        actions.send_keys(Keys.ENTER)
        driver.find_element(By.XPATH, "//*[@id='MainContent_tbxMRNo']").clear()
        driver.find_element(By.XPATH, "//*[@id='MainContent_tbxBirthday']").clear()
        driver.find_element(By.XPATH, "//*[@id='MainContent_tbxVCode']").clear()
        driver.find_element(By.XPATH, "//*[@id='MainContent_btnReflash']").click()
        time.sleep(0.3)
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
df_temp = pd.DataFrame({'X_train': X_train, 'y_train': y_train})
df_temp.to_csv('training_data.csv')

In [322]:
15195/(5071*5)#實際上有辨識出來的張數/總共的張數

0.5992900808519029

In [324]:
4586/(1523*5)#實際上有辨識出來的張數/總共的張數

0.6022324359816152

In [336]:
20150/(6765*5)

0.5957132298595713